# The tool of  exploratory data analysis: descriptive statistics
1. Examination of frenquency distribution:<br>
c1 = df['column'].value_counts(sort=False, normalize=True, dropna=False)<br>
c1 = df['column'].value_counts(sort=False)
2. Graphical representations of variables of interest<br>
ax=sns.countplot(x='SKIPCAT', data=sub)<br>
ax1=sns.distplot(sub['SKIPTOTAL'].dropna(), kde=False)<br>
ax2=sns.factorplot(x="H1GH2", y="SKIPCAT", data=sub, kind="bar", ci=None)<br>
ax3=sns.regplot(x="H1DA10", y="SKIPTOTAL", fit_reg=False, data=sub)<br>
3. Calculations of center and spread<br>
mean, median, mode<br>
sub['SKIPTOTAL'].describe()<br>
sub['SKIPTOTAL'].mean()<br>
sub['SKIPTOTAL'].std()<br>
sub['SKIPTOTAL'].median()<br>
sub['SKIPTOTAL'].mode()<br>
sub['SKIPTOTAL'].min()<br>
sub['SKIPTOTAL'].max()<br>
4. OLS:<br>
import statsmodels.formula.api as smf<br>
model1 = smf.ols(formula = 'SKIPTOTAL ~ C(HEADCAT)', data = sub)<br>
result1 = model1.fit()<br>
result1.summary()<br>
5. Post Hoc Test: Tukey's<br>
import statsmodels.stats.multicomp as multi<br>
mc = multi.MultiComparison(sub1['SKIPTOTAL'], sub1['H1GH2']).tukeyhsd()<br>

In [1]:
import numpy as np
import pandas as pd

## load data
data = pd.read_csv("E:/Jupyter/DATA _ANALYST/Data_Management_and_Visualization/data/addhealth.csv", low_memory=False)
pd.set_option("display.float_format", lambda x: '%f'%x)
## convert column names to upper case
data.columns = map(str.upper, data.columns)

In [2]:
## remove the limit and allows all rows and columns to be displayed
pd.set_option('display.max_columns',None)

In [3]:
## only keep the variables that will be analysis
sub1 = data[['AID', 'H1GH2', 'H1ED1', 'H1ED2', 'H1GI1Y']]
sub2 = sub1[(sub1['H1GI1Y'] >= 80)]
sub = sub2.copy()
sub.head()

,AID,H1GH2,H1ED1,H1ED2,H1GI1Y
5,57104649,1,2,0,81
6,57104676,0,1,0,83
7,57109625,1,3,4,81
8,57110897,1,2,10,81
9,57111071,2,2,0,81


In [4]:
## convert object to numeric
sub['H1GH2'] = pd.to_numeric(sub['H1GH2'], errors='coerce')
sub['H1ED1'] = pd.to_numeric(sub['H1ED1'], errors='coerce')
sub['H1ED2'] = pd.to_numeric(sub['H1ED2'], errors='coerce')

In [5]:
## coding out missing data
sub['H1GH2'] = sub['H1GH2'].replace([6,8], np.nan)
sub['H1ED1'] = sub['H1ED1'].replace([6, 7, 8, 9], np.nan)
sub['H1ED2'] = sub['H1ED2'].replace([996, 997, 998], np.nan)

In [6]:
sub['H1ED1'].value_counts(dropna=False)

2.000000    1156
1.000000     805
3.000000     301
0.000000     287
nan           25
Name: H1ED1, dtype: int64

In [7]:
## create secondary variables
def times_skip(row):
    if row['H1ED1']==0:
        return 0
    elif row['H1ED1']==1:
        return 2
    elif row['H1ED1']==2:
        return 7
    elif row['H1ED1']==3:
        return 45
    else:
        return np.nan
    
sub['SKIPFREQ'] = sub.apply(lambda row: times_skip(row), axis=1)

In [8]:
def total_skip(row):
    if row['SKIPFREQ'] != np.nan and row['H1ED2'] != np.nan:
        return row['SKIPFREQ']+row['H1ED2']
    elif row['SKIPFREQ'] == np.nan and row['H1ED2'] != np.nan:
        return row['H1ED2']
    elif row['SKIPFREQ'] != np.nan and row['H1ED2'] == np.nan:
        return row['SKIPFREQ']
    else:
        return np.nan
    
sub['SKIPTOTAL'] = sub.apply(lambda row: total_skip(row), axis=1)

In [9]:
## create secondary variables
def head_level(row):
    if row['H1GH2']==0:
        return 0
    elif row['H1GH2']==1:
        return 0
    elif row['H1GH2']==2:
        return 0
    elif row['H1GH2']==3:
        return 1
    elif row['H1GH2']==4:
        return 1
    else:
        return np.nan
    
sub['HEADCAT'] = sub.apply(lambda row: head_level(row), axis=1)

In [10]:
sub['HEADCAT'].value_counts()

0.000000    2402
1.000000     168
Name: HEADCAT, dtype: int64

# Ordinary Lease Squares: OLS

### Explanatory variables with two levels

In [33]:
## explainatory: HEADCAT -> response: SKIPTOTAL
import statsmodels.formula.api as smf

model1 = smf.ols(formula = 'SKIPTOTAL ~ C(HEADCAT)', data = sub)
result1 = model1.fit()
result1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SKIPTOTAL   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     14.77
Date:                Thu, 05 Apr 2018   Prob (F-statistic):           0.000124
Time:                        16:10:17   Log-Likelihood:                -10367.
No. Observations:                2545   AIC:                         2.074e+04
Df Residuals:                    2543   BIC:                         2.075e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             9.5628      0.291     32.811      0.000       8.991      10.134
C(HEADCAT)[T.1.0]     4.4128      1.148      3.844      0.000       2.161       6.664
==============================================================================
Omnibus:                     1147.631   Durbin-Watson:                   2.003
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4495.006
Skew:                           2.286   Prob(JB):                         0.00
Kurtosis:                       7.636   Cond. No.                         4.09
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [34]:
## generated mean values based on the explanatory variable
sub2 = sub[['HEADCAT', 'SKIPTOTAL']].dropna()
sub2.groupby('HEADCAT').mean()

,SKIPTOTAL
HEADCAT,
0.000000,9.562789
1.000000,13.975610


### Explanatory variables with more than two levels
**Here we choose the *Tukey's Honestly Significant Difference Test* as my Post Hoc Test**

In [11]:
import statsmodels.formula.api as smf

# create a new sub dataframe
sub1 = sub[['H1GH2', 'SKIPTOTAL']].dropna()

In [12]:
## explainatory: H1GH2 -> response: SKIPTOTAL
## ~ tilde
model2 = smf.ols(formula = 'SKIPTOTAL ~ C(H1GH2)', data = sub1).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SKIPTOTAL   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     9.483
Date:                Thu, 05 Apr 2018   Prob (F-statistic):           1.31e-07
Time:                        16:21:17   Log-Likelihood:                -10355.
No. Observations:                2545   AIC:                         2.072e+04
Df Residuals:                    2540   BIC:                         2.075e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           7.3511      0.944      7.784      0.000       5.499       9.203
C(H1GH2)[T.1.0]     1.7289      1.009      1.714      0.087      -0.249       3.707
C(H1GH2)[T.2.0]     4.4909      1.119      4.013      0.000       2.297       6.685
C(H1GH2)[T.3.0]     6.1060      1.525      4.004      0.000       3.116       9.096
C(H1GH2)[T.4.0]     9.6489      3.042      3.172      0.002       3.684      15.614
==============================================================================
Omnibus:                     1135.655   Durbin-Watson:                   2.008
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4389.720
Skew:                           2.264   Prob(JB):                         0.00
Kurtosis:                       7.571   Cond. No.                         13.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
## generated mean values based on the explanatory variable
sub1.groupby('H1GH2').mean()

,SKIPTOTAL
H1GH2,
0.000000,7.351111
1.000000,9.080050
2.000000,11.842011
3.000000,13.457143
4.000000,17.000000


In [14]:
sub1.groupby('H1GH2').std()

,SKIPTOTAL
H1GH2,
0.000000,11.562961
1.000000,13.466404
2.000000,15.984260
3.000000,16.695052
4.000000,19.956474


In [15]:
import statsmodels.stats.multicomp as multi

mc = multi.MultiComparison(sub1['SKIPTOTAL'], sub1['H1GH2']).tukeyhsd()
mc.summary()

group1,group2,meandiff,lower,upper,reject
0.0,1.0,1.7289,-1.0244,4.4822,False
0.0,2.0,4.4909,1.4364,7.5454,True
0.0,3.0,6.106,1.9436,10.2685,True
0.0,4.0,9.6489,1.3454,17.9524,True
1.0,2.0,2.762,0.8594,4.6645,True
1.0,3.0,4.3771,0.9689,7.7852,True
1.0,4.0,7.9199,-0.0322,15.8721,False
2.0,3.0,1.6151,-2.0407,5.2709,False
2.0,4.0,5.158,-2.9034,13.2194,False
3.0,4.0,3.5429,-5.0001,12.0858,False
